# Algorithme du gradient conjugué

In [1]:
using LinearAlgebra
using Optim

Résolvez
$$
\min_x f(x) = \frac{1}{2} x^T A x + b^T x + a
$$
où $A \succ 0$. En posant $\nabla f(x) = 0$, c'est équivalent à résoudre le système linéaire $Ax = -b$.

Construisons la fonction quadratique associée au programme précédent.

In [2]:
f = x -> 0.5*dot(x,A*x)+dot(b,x)

#1 (generic function with 1 method)

## Un exemple simple

Adapté de https://www.rose-hulman.edu/~bryan/lottamath/congrad.pdf

Soit
$$
A =
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
$$
Considérons la fonction à minimiser
$$
f(x) = \frac{1}{2} x^TAx,
$$
et supposons que nous avons déjà calculer
\begin{align*}
d_0 &= (1, 0, 0)\\
d_1 & = (1, −3, 0)\\
d_2 &= (−2, 6, −5).
\end{align*}

Vérifions que $d_0$, $d_1$ et $d_2$ sont $A$-conjugés.

In [3]:
A = [ 3.0 1 0 ; 1 2 2 ; 0 2 4]
d0 = [ 1.0 0 0 ]'
d1 = [ 1.0 -3.0 0.0 ]'
d2 = [ -2.0 6.0 -5.0]'

println("$(dot(d0, A*d1)) $(dot(d0, A*d2)) $(dot(d1, A*d2))")

0.0 0.0 0.0


Prenons comme solution initiale $x_0 = (1, 2, 3)$. Calculons $x_1$, $x_2$ et $x_3$ en utilisant l'algorithme du gradient conjugué. $x_3$ est-il optimal?

$$
\nabla f(x) = Ax
$$

In [4]:
x0 = [1 2 3.0]'
-A*x0

3×1 Array{Float64,2}:
  -5.0
 -11.0
 -16.0

In [5]:
f = x -> dot(x,A*x)

#3 (generic function with 1 method)

Nous devons calculer $\alpha_k$, $k = 1,2,3$, en résolvant
$$
\min_{\alpha} f(x_k + \alpha d_k)
$$

Afin d'obtenir $\alpha_0$, nous devons minimiser
\begin{align*}
f(x_0 + \alpha d_0) &= \frac{1}{2}
\left(\begin{pmatrix} 1 & 2 & 3\end{pmatrix} + \alpha \begin{pmatrix} 1 & 0 & 0\end{pmatrix} \right)
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
\left(\begin{pmatrix} 1 \\ 2 \\ 3 \end{pmatrix} + \alpha \begin{pmatrix} 1 \\0 \\0 \end{pmatrix} \right)
\\
& = \frac{1}{2}\begin{pmatrix} 1 + \alpha & 2 & 3 \end{pmatrix}
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
\begin{pmatrix} 1 + \alpha \\ 2 \\ 3 \end{pmatrix}\\
& = \frac{1}{2}\begin{pmatrix} 1 + \alpha & 2 & 3 \end{pmatrix}
\begin{pmatrix} 5+3\alpha \\ 11+\alpha \\ 16 \end{pmatrix}\\
& = \frac{1}{2}
((1 + \alpha)(5+3\alpha) + 22+2\alpha + 48 ) \\
& = \frac{1}{2}
( 3\alpha^2 + 8\alpha + 5 + 70 + 2\alpha ) \\
& = \frac{3}{2}\alpha^2 + 5\alpha+\frac{75}{2}
\end{align*}
par rapport à $\alpha$.

Nous pouvons l'obtenir en cherchant le zéro de la dérivée par rapport à $\alpha$, c'est-à-dire
$$
\frac{d}{d\alpha} f(x+\alpha d) = 0,
$$
ou

$$
d^T \nabla f(x+\alpha d) = 0
$$

Dès lors, nous devons avoir

$$
3\alpha + 5 = 0
$$
Ainsi,
$$
\alpha_{0} = -\frac{5}{3}
$$
$$
x_1 = x_0 - \frac{5}{3} d_0 = \begin{pmatrix} -\frac{2}{3} \\ 2 \\ 3  \end{pmatrix}
$$

Nous pouvons aussi directement calculer $\alpha_0$ comme
$$
\alpha_0 = - \frac{d_0^T\nabla f(x_0)}{d_0^TAd_0}
$$

In [6]:
x0 = [1 ; 2 ; 3.0]
∇f = x -> A*x

#5 (generic function with 1 method)

In [7]:
d0 = [1 ; 0 ; 0]
α0 = -dot(d0,∇f(x0))/dot(d0,A*d0)

-1.6666666666666667

In [8]:
x1 = x0+α0*d0

3-element Array{Float64,1}:
 -0.6666666666666667
  2.0
  3.0

Une recherche linéaire à partir de $x_1$ dans la direction $d_1$ exige de minimiser
$$
f(x_1 + \alpha d_1) = \frac{15}{2}\alpha^2 - 28\alpha + \frac{100}{3},
$$
ce qui a lieu en
$$
\alpha_1 = \frac{28}{15},
$$
donnant
$$
x_2 = x_1 + \frac{28}{15}d_1 =
    \begin{pmatrix}
     \frac{6}{5} \\ \frac{-18}{5} \\ 3
    \end{pmatrix}.
$$

In [9]:
α1 = -dot(d1,A*x1)/dot(d1,A*d1)

1.8666666666666665

In [10]:
28/15

1.8666666666666667

In [11]:
x2 = x1+α1*d1

3×1 Array{Float64,2}:
  1.1999999999999997
 -3.5999999999999996
  3.0

La recherche linéaire finale à partir de $x_2$ dans la direction $d_2$ requiert de minimiser
$$
f(x_2 + \alpha d_2) = 20 \alpha^2 - 24\alpha + \frac{36}{5},
$$
ce qui a lieu en
$$
\alpha_2 = \frac{3}{5},
$$
donnant
$$
x_3 = x_2 + \frac{3}{5}d_2 =
    \begin{pmatrix}
     0 \\ 0 \\ 0
    \end{pmatrix},
$$
ce qui est bien entendu correct.

Similairement, nous pouvons calculer le nouveau point comme

In [12]:
α2 = -dot(d2,A*x2)/dot(d2,A*d2)
x3 = x2+α2*d2

3×1 Array{Float64,2}:
 -4.440892098500626e-16
  8.881784197001252e-16
 -4.440892098500626e-16

## Une implémentation naïve

Une première version de l'algorithme du gradient conjugué suit.

In [13]:
function cg_quadratic(A:: Matrix, b:: Vector, x0:: Vector, trace:: Bool = false)
    n = length(x0)
    x = x0
    g = b+A*x
    d = -g
    if (trace)
        iter = [ x ]
        iterg = [ norm(g) ]
        iterd = [ norm(d) ]
    end
    k = 0
    
    for k = 1:n-1
        Ad = A*d
        normd = dot(d,Ad)
        α = -dot(d,g)/normd
        x += α*d
        if (trace)
            iter = [ iter; [x] ]
            iterg = [ iterg; norm(g)]
            iterd = [ iterd; norm(d) ]
        end
        g = b+A*x
        β = dot(g,Ad)/normd
        d = -g+β*d
    end

    normd = dot(d,A*d)
    α = -dot(d,g)/normd
    x += α*d
    if (trace)
        g = b+A*x # g must be equal to 0
        iter = [ iter; [x] ]
        iterg = [ iterg; norm(g)]
        iterd = [ iterd; norm(d) ]
        return x, iter, iterg, iterd
    end
    
    return x
end

cg_quadratic (generic function with 2 methods)

Considérons l'exemple simple

In [14]:
A = [2 1; 1 2]
b = [1, 0]
A\(-b)

2-element Array{Float64,1}:
 -0.6666666666666666
  0.3333333333333333

Nous voulons résoudre
$$
    \min_{\alpha} f(x) = \frac{1}{2}x^TAx+b^Tx+c
$$

Ou, de manière équivalente, nous résolvons
$$
    c+\min_{\alpha} f(x) = \frac{1}{2}x^TAx+b^Tx
$$

In [15]:
cg_quadratic(A, b, [0, 0], true)

([-0.6666666666666666, 0.3333333333333333], [[0.0, 0.0], [-0.5, 0.0], [-0.6666666666666666, 0.3333333333333333]], [1.0, 1.0, 0.0], [1.0, 1.0, 0.5590169943749475])

Que se passe-t-il si $A$ c'est pas définie positive?

In [16]:
A = [ 1 2 ; 2 1]
A\(-b)

2-element Array{Float64,1}:
  0.3333333333333333
 -0.6666666666666666

In [17]:
cg_quadratic(A, b, [0, 0], true)

([0.33333333333333326, -0.6666666666666666], [[0.0, 0.0], [-1.0, 0.0], [0.33333333333333326, -0.6666666666666666]], [1.0, 1.0, 1.1102230246251565e-16], [1.0, 1.0, 4.47213595499958])

In [18]:
det(A)

-3.0

In [19]:
eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -1.0
  3.0
vectors:
2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [20]:
cg_quadratic(A, b, [1, 1], true)

([0.3333333333333335, -0.6666666666666667], [[1.0, 1.0], [-0.36986301369863006, -0.02739726027397249], [0.3333333333333335, -0.6666666666666667]], [5.0, 5.0, 2.220446049250313e-16], [5.0, 5.0, 0.9763790695367754])

In [21]:
f([1/3,-2/3])

-0.3333333333333333

In [22]:
f([0,0])

0

The conjugate gradient finds the solution of the linear system, and this does correspond to a first-order critical point of the function.

In [ ]:
∇f = x -> A*x+b

In [ ]:
x = [1.0/3; -2.0/3]
∇f(x)

In [ ]:
x = [1; 1]
∇f(x)

In [ ]:
step= x -> x-α*∇f(x)

In [ ]:
α = 10
dot(step(x),A*step(x))

In [ ]:
λ, u = eigen(A)

In [ ]:
u

In [ ]:
x = u[:,1]
α = 10
f = x -> 0.5*dot(x,A*x)+dot(b,x)
f(step(x))

In [ ]:
α = 1000
dot(step(x),A*step(x))+dot(b,x)

In [ ]:
f(x)

In [ ]:
x = [1/3.0; -2/3]
f(x)

In [ ]:
cg_quadratic(A, b, x, true)

We will need to incorporate a test on $\nabla f(x_k)$!

In [ ]:
A = [ 1 2 ; 0 4 ]
eigen(A)

In [ ]:
eigen(A*A')

In [ ]:
A = [ 3 1; 1 2 ]
eigen(A)

In [ ]:
eigen(A*A')

A more complex example

In [ ]:
n = 500;
m = 600;
A = randn(n,m);
A = A * A';  # A is now a positive semi-definite matrix
A = A+I # A is positive definite
b = zeros(n)
for i = 1:n
  b[i] = randn()
end
x0 = zeros(n)

In [ ]:
b1 = A\(-b)

In [ ]:
b2, iter, iterg, iterd = cg_quadratic(A, b, x0, true);

In [ ]:
norm(b1-b2)

In [ ]:
iterg

In [ ]:
iterd

It works, but do we need to perform all the 500 iterations? We could be satisfied if we are close to the solution. We can mesure the residual of the linear system
$$
r = b+Ax
$$
that is nothing else the the gradient of the objective function of the quadratic optimization problem.

In [ ]:
iter

We incorporate a convergence test in the function.

In [ ]:
function cg_quadratic_tol(A:: Matrix, b:: Vector, x0:: Vector, trace:: Bool = false, tol = 1e-8)
    n = length(x0)
    x = x0
    if (trace)
        iter = [ x ]
    end
    g = b+A*x
    d = -g
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    while ((dot(g,g) > tol2) && (k <= n))
        Ad = A*d
        normd = dot(d,Ad)
        α = dot(g,g)/normd
#        α = -dot(d,g)/normd
        x += α*d
        if (trace)
            iter = [ iter; x ]
        end
        g = b+A*x
        β = dot(g,Ad)/normd
        d = -g+β*d
        k += 1
    end

    if (trace)
        iter = [ iter; x ]
        return x, iter, k
    end

    return x, k
end

In [ ]:
x, iter, k = cg_quadratic_tol(A, b, x0, true)

The number of iterations is

In [ ]:
k

Are we close to the solution?

In [ ]:
norm(b1-x)

In [ ]:
size(A)

which is much less than the problem dimension

## Preconditioned conjugate gradient

A basic implementation of a preconditioned conjugate gradient algorithm follows, were $M$ is the inverse of the preconditioner to apply.

In [ ]:
function pcg_quadratic_tol(A:: Matrix, b:: Vector, x0:: Vector, M:: Matrix,
                           trace:: Bool = false, tol = 1e-8)
    n = length(x0)
    x = x0
    if (trace)
        iter = [ x ]
    end
    g = b+A*x
    v = M*g
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k <= n))
#    while ((dot(g,g) > tol2) && (k <= n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = [ iter; x ]
        end
        g += α*Ad
        v = M*g
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        iter = [ iter; x ]
        return x, iter, k
    end

    return x, k
end

Let's check first that when there is no preconditioning, we obtain the same iterates.
Set

In [ ]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

In [ ]:
k, norm(x-b1)

We can compute the eigenvalues and condition number of $A$.

In [ ]:
eigen(A)

In [ ]:
cond(A)

Try to compute a simple precontionner using the inverse of the diagonal of matrix $A$.

In [ ]:
D = 1 ./diag(A)
M = Diagonal(D)

Unfortunately, in this case, it does not help as the condition number is not improving.

In [ ]:
B = M*A
cond(B)

Consider another situation when $A$ is diagonal.

In [ ]:
n = 1000;
A = zeros(n,n);
for i = 1:n
    A[i,i] = 10*rand()
end
b = zeros(n)
for i = 1:n
  b[i] = rand()
end
x0 = zeros(n)
cond(A)

The solution we are looking for is

In [ ]:
A\b

Without preconditionning, with have the iterates sequence

In [ ]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

This is equivalent to the unpreconditioned version.

In [ ]:
x, iter, k = cg_quadratic_tol(A, b, x0, true)

However, since $A$ is diagonal, an obvious diagonal preconditionner is $A^{-1}$ itself.

In [ ]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end

The condition number of the preconditioned matrix is of course equal to 1.

In [ ]:
cond(M*A)

The theory then predicts that we converge in one iteration with the precionditionned conjugate gradient.

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

Consider now another example.

In [ ]:
A = zeros(n,n)+3*I
for i = 1:n-1
    A[i,i+1] = 1.4
    A[i+1,i] = 1.4
end
A

In [ ]:
eigen(A)

In [ ]:
A\(-b)

In [ ]:
x, iter, k = cg_quadratic_tol(A, b, x0, true)

In [ ]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end

In [ ]:
cond(A)

In [ ]:
cond(M*A)

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

There is no advantage.

In [ ]:
M = A^(-1)

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

Consider now the following example.

In [ ]:
n = 1000
A = zeros(n,n)+Diagonal([2+i*i for i=1:n])

In [ ]:
for i = 1:n-1
    A[i,i+1] = 1
    A[i+1,i] = 1
end
A[n,1] = 1
A[1,n] = 1
cond(A)

In [ ]:
κ = cond(A)
(sqrt(κ)-1)/(sqrt(κ)+1)

In [ ]:
A

In [ ]:
A^(-1)

In [ ]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

In [ ]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end
cond(A*M), cond(A)

In [ ]:
M

In [ ]:
A*M

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

In [ ]:
function pcg_quadratic(A:: Matrix, b:: Vector, x0:: Vector, M:: Matrix,
                       trace:: Bool = false, tol = 1e-8)
    n = length(x0)
    x = x0
    if (trace)
        iter = [ x ]
    end
    g = b+A*x
    v = M\g
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k <= n))
#    while ((dot(g,g) > tol2) && (k <= n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = [ iter; x ]
        end
        g += α*Ad
        v = M\g
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        iter = [ iter; x ]
        return x, iter, k
    end

    return x, k
end

In [ ]:
function ichol(A:: Matrix)

    n = size(A,1)
    C = zeros(n,n)+I
    
    for k=1:n
        C[k,k] = sqrt(A[k,k])
        for i=(k+1):n
            if (A[i,k] != 0)
                C[i,k] = A[i,k]/A[k,k]    
            end
        end
        for j=(k+1):n
            for i=j:n
                if (A[i,j] != 0)
                    C[i,j] = A[i,j]-A[i,k]*A[j,k]
                end
            end
        end
    end

    return C
end

In [ ]:
C = cholesky(A)
C.L

In [ ]:
M = C.L*C.U

In [ ]:
x, iter, k = pcg_quadratic(A, b, x0, M, true)

In [ ]:
C = ichol(A)

In [ ]:
M=C*C'

In [ ]:
x, iter, k = pcg_quadratic(A, b, x0, M, true)

An efficient implementation would make use of sparse matrices and specific functions to compute v.